# The Python `geopack` (with Tsyganenko Models)

The `geopack` are available in Fortran at https://ccmc.gsfc.nasa.gov/modelweb/magnetos/data-based/Geopack_2005.html and its DLM in IDL is available at http://ampere.jhuapl.edu/code/idl_geopack.html. As the necessary parts of `geopack`, the Tsyganenko models are available in Fortran at https://ccmc.gsfc.nasa.gov/models/modelinfo.php?model=Tsyganenko%20Magnetic%20Field.

Now I have converted the `geopack` and the Tsyganenko models (T89, T96, T01, and T04) to Python. All these codes will be referred to as the Python version of `geopack`. The current version is a literal translation of the Fortran counterparts, except that `recalc` and `sun` are re-written, and that `igrf_geo` and `igrf_gsm` are re-organized. These changes are explained in the next section, followed by a section containing test results.


## Changes in the Python `geopack`

#### The Python `recalc`
The Fortran `recalc` contains an unnecessary long list of IGRF coefficients and many `goto` statements which are also unecessary. In the Python `recalc`, we load the IGRF coefficients from `igrf12coeffs.txt`, available at https://www.ngdc.noaa.gov/IAGA/vmod/igrf12coeffs.txt. By loading the coefficients automatically, `recalc` is greatly simplified. And the limitation on input time in the Fortran version is removed by smart interpolation and extrapolation.

#### The Python `sun`
The Fortran `sun` contains many "magic numbers", which are now replaced by equations at known references. The blocks on calculating the solar parameters follow the notation and equations at http://aa.usno.navy.mil/faq/docs/SunApprox.php. And the blocks on calculating the Greenwich mean sidereal time (GMST) follow the notation and equations at http://aa.usno.navy.mil/faq/docs/GAST.php. This procedure should accept an array of input times, enabling a bulk coordinate transformation.

#### The Python `igrf_geo` and `igrf_gsm`
The Fortran `igrf_geo` contains complex `goto` structures and is not very intuitive. I've re-written the code and added many comments. For more details, please refer to the exellent note at https://hanspeterschaub.info/Papers/UnderGradStudents/MagneticField.pdf. The Fortran `igrf_gsm` essentially repeats the whole procedure of `igrf_geo`. This is a perfect situation for code reuse. So I changed the `igrf_gsm` as a wrapper of `igrf_geo` with the proper coordinate transforms.

## Tests on the Python `geopack`

In this section, We test the results from the Python `geopack` against those from the Fortran `geopack`. In case people want to replicate the tests, the Appendix includes a brief instruction on running `geopack` in Fortran.

Tsyganenko models are tested first, because although they are parts of the Python `geopack`, these models run independently of the Fortran `geopack`.

The core of the Fortran `geopack` is `recalc` and `sun`, which prepare all basic information for later calculations. They should be tested first and the most. The rest of the Fortran `geopack` consists of several groups:
1. Models for the internal magnetic field (`dip`, `igrf_geo`, `igrf_gsm`)
2. Functions for coordinate transforms among (`geo`,`mag`,`sm`,`gsm`,`gse`) and the related math functions (`sphcar`,`bspcar`,`bcarsp`)
3. Functions for tracing (`trace`, `step`, `rhand`), and models for the magnetopause (`shuetal_mgnp`,`t96_mgnp`).

These parts will be tested one by one in this section.

#### External magnetic fields: Tsyganenko models

The external magnetic fields are from the Tsyganenko models. These models run on their own.

The test inputs are:
* Time/dipole tilt: 2001-01-01/02:03:04 UT/`ps = -0.533585131 rad`
* Position: `(xgsm,ygsm,zgsm) = (-5.1,0.3,2.8)`
* Model parameter:
    * T89: `iopt = 2`
    * T96, T01, and T04: `par = [2., -87., 2., -5., 0., 0., ps, xgsm,ygsm,zgsm]`.

The model magnetic field (internal field is not included) `(bxgsm,bygsm,bzgsm)` are:

|<p align="left"> Model| <p align="left"> Version | <p align="left"> GSM Bx (nT)| <p align="left"> GSM By (nT)  | <p align="left"> GSM Bz (nT) |
|---|---|---|---|---|
| <p align="left"> T89 | <p align="left"> Fortran | <p align="left"> 20.7721329 | <p align="left"> -0.646554828 | <p align="left"> -15.0716438 |
|                      | <p align="left"> Python  | <p align="left"> 20.77213175686351 | <p align="left"> -0.6465547428023687 | <p align="left"> -15.071641970338984 |
|                      | <p align="left"> IDL DLM | <p align="left"> 20.772132  | <p align="left"> -0.64655478  | <p align="left"> -15.071642  |
| <p align="left"> T96 | <p align="left"> Fortran | <p align="left"> 61.1784935 | <p align="left"> -1.46160448  | <p align="left"> -40.4486618 |
|                      | <p align="left"> Python  | <p align="left"> 61.178346985193215| <p align="left"> -1.4611972499959456 | <p align="left"> -40.44976904223083  |
|                      | <p align="left"> IDL DLM | <p align="left"> 61.178075  | <p align="left"> -1.4611772   | <p align="left"> -40.448863  |
| <p align="left"> T01 | <p align="left"> Fortran | <p align="left"> 46.3536224 | <p align="left"> 1.43994296   | <p align="left"> -32.0043602 |
|                      | <p align="left"> Python  | <p align="left"> 46.35361850144623 | <p align="left"> 1.4399149705997756  | <p align="left"> -31.998997670712665 |
|                      | <p align="left"> IDL DLM | <p align="left"> 46.972664  | <p align="left"> 1.5442350    | <p align="left"> -31.354185  |
| <p align="left"> T04 | <p align="left"> Fortran | <p align="left"> 10.252864409458805| <p align="left"> -2.8946251063460302 | <p align="left"> -9.6220929668066475 |
|                      | <p align="left"> Python  | <p align="left"> 10.251787900560483| <p align="left"> -2.894661363001944  | <p align="left"> -9.615717866328614  |
|                      | <p align="left"> IDL DLM | <p align="left"> 12.009174  | <p align="left"> -2.6345250   | <p align="left"> -11.963656  |

#### `recalc` and `sun`

Since this routine is the core of `geopack` and I've made changes to it. I'll test some internal steps. The routine does the following things that will be tested.

1. Loads IGRF coefficients for a given time and applies normalization coefficients to the coefficients

The IGRF coefficients after being interpolated to the given time are:

|<p align="left"> Version | <p align="left"> g[1] | <p align="left"> g[11]  | <p align="left"> g[21] | <p align="left"> h[2] | <p align="left"> h[12]  | <p align="left"> h[22] | 
|---|---|---|---|
| <p align="left"> Fortran |<p align="left"> -29606.8828     | <p align="left"> 789.080017   | <p align="left"> 72.4200058  | <p align="left"> 5164.88037    |<p align="left"> -230.680008   |<p align="left"> -17.9600010  |
| <p align="left"> Python  |<p align="left"> -29606.42169927 | <p align="left"> 789.03618706 | <p align="left"> 72.56048774 | <p align="left"> 5164.43743875 |<p align="left"> -230.56349752 |<p align="left"> -17.98709929 |
    
The results are slightly different because the interpolation in the Fortran version is not very accurate: the given time is truncated to the start of the day. However, in the Python version, I use the given time up to milli second resolution.

The IGRF coefficients after normalizations are applied are:

|<p align="left"> Version | <p align="left"> g[1] | <p align="left"> g[11]  | <p align="left"> g[21] | <p align="left"> h[2] | <p align="left"> h[12]  | <p align="left"> h[22] | 
|---|---|---|---|
| <p align="left"> Fortran |<p align="left"> -29606.8828     | <p align="left"> 4366.75781    | <p align="left"> 1045.56384    | <p align="left"> 5164.88037    |<p align="left"> -902.678345   |<p align="left"> -339.500122   |
| <p align="left"> Python  |<p align="left"> -29606.42169927 | <p align="left"> 4366.51513798 | <p align="left"> 1047.59204175 | <p align="left"> 5164.43743875 |<p align="left"> -902.22239376 |<p align="left"> -340.01238166 |

2. Calculates parameters related to the Sun
Since I've rewritten `sun`, here is a test on its return values:

|<p align="left"> Version | <p align="left"> gmst (rad) | <p align="left"> slong (rad) | <p align="left"> srasn (rad) | <p align="left"> sdec (rad) |
|---|---|---|---|
| <p align="left"> Fortran |<p align="left"> 2.29623127        | <p align="left"> 4.89966297        | <p align="left"> 4.91595507        | <p align="left"> -0.401530176         |
| <p align="left"> Python  |<p align="left"> 2.296253490174557 | <p align="left"> 4.899558241818756 | <p align="left"> 4.915948604659666 | <p align="left"> -0.40152585209539443 |

3. Prepare rotation angles for coordinate transforms
These angles will be tested when we test coordinate transforms.




#### Internal magnetic fields: IGRF and dipole

The internal magnetic fields are from the IGRF and dipole models. They are in the Fortran `geopack` and depend on `recalc`.

The test inputs are:
* time/dipole tilt: 2001-01-01/02:03:04 UT/`ps = -0.533585131 rad`
* position: `(xgsm,ygsm,zgsm) = (-5.1,0.3,2.8)`

The IGRF and dipole magnetic fields are:

| Model |<p align="left"> Version | <p align="left"> GSM Bx (nT) | <p align="left"> GSM By (nT) | <p align="left"> GSM Bz (nT) |
|---|---|---|---|---|
| <p align="left"> Dipole | <p align="left"> Fortran |<p align="left"> 266.046814         | <p align="left"> -20.2048283         | <p align="left"> -57.4973526         |
|                         | <p align="left"> Python  |<p align="left"> 266.04028284777775 | <p align="left"> -20.204186166677108 | <p align="left"> -57.492114467356956 |
|                         | <p align="left"> IDL DLM |<p align="left"> 266.04064          | <p align="left"> -20.204224          | <p align="left"> -57.492450          |
| <p align="left"> IGRF   | <p align="left"> Fortran |<p align="left"> 262.836670         | <p align="left"> -19.3072014         | <p align="left"> -50.3504410         |
|                         | <p align="left"> Python  |<p align="left"> 262.8292494578462  | <p align="left"> -19.305779063359893 | <p align="left"> -50.34573331501855  |
|                         | <p align="left"> IDL DLM |<p align="left"> 262.82959          | <p align="left"> -19.305938          | <p align="left"> -50.346050          |

Note that we are testing the Python `igrf_gsm`, which is a wrapper of `igrf_geo` and uses `sphcar`, `bspcar` and `bcarsp`. Therefore the test includes the test for these functions.

#### Coordinate transforms

The coordinate transforms are being tested with the following inputs:
* Time/dipole tilt: 2001-01-01/02:03:04 UT/`ps = -0.533585131 rad`
* Position: `(xgsm,ygsm,zgsm) = (-5.1,0.3,2.8)`

Here are its transforms in all the supported coordinates:

| Coord |<p align="left"> Version | <p align="left"> X (Re) | <p align="left"> Y (Re) | <p align="left"> Z (Re) |
|---|---|---|---|---|
| <p align="left"> GSM | <p align="left"> Input   |<p align="left"> -5.1 | <p align="left"> 0.3 | <p align="left"> 2.8 |
| <p align="left"> GEO | <p align="left"> Fortran |<p align="left"> 2.80123758           | <p align="left"> -2.40482044        | <p align="left"> 4.50665092         |
|                      | <p align="left"> Python  |<p align="left"> 2.8011944117533565   | <p align="left"> -2.4048913761357267| <p align="left"> 4.5066403602406275 |
|                      | <p align="left"> IDL DLM |<p align="left"> 2.8012418            | <p align="left"> -2.4048176         | <p align="left"> 4.5066502          |
| <p align="left"> GSE | <p align="left"> Fortran |<p align="left"> -5.09999990          | <p align="left"> 0.905646503        | <p align="left"> 2.66642165         |
|                      | <p align="left"> Python  |<p align="left"> -5.1                 | <p align="left"> 0.9057902360912311 | <p align="left"> 2.6663726513864643 |
|                      | <p align="left"> IDL DLM |<p align="left"> -5.0999999           | <p align="left"> 0.90564978         | <p align="left"> 2.6664205          |
| <p align="left"> SM  | <p align="left"> Fortran |<p align="left"> -2.96689892          | <p align="left"> 0.300000012        | <p align="left"> 5.00474834         |
|                      | <p align="left"> Python  |<p align="left"> -2.9670092644479498  | <p align="left"> 0.3                | <p align="left"> 5.004683409035982  |
|                      | <p align="left"> IDL DLM |<p align="left"> -2.9670018           | <p align="left"> 0.30000001         | <p align="left"> 5.0046877          |
| <p align="left"> MAG | <p align="left"> Fortran |<p align="left"> 2.29868531           | <p align="left"> 1.89961421         | <p align="left"> 5.00474834         |
|                      | <p align="left"> Python  |<p align="left"> 2.298686529948157    | <p align="left"> 1.8997853069109853 | <p align="left"> 5.004683409035982  |
|                      | <p align="left"> IDL DLM |<p align="left"> 2.2986287            | <p align="left"> 1.8998436          | <p align="left"> 5.0046877          |
| <p align="left"> GEI | <p align="left"> Fortran |<p align="left"> -0.0591988564        | <p align="left"> 3.69142103         | <p align="left"> 4.50665092         |
|                      | <p align="left"> Python  |<p align="left"> -0.05919908606124502 | <p align="left"> 3.691434427381819  | <p align="left"> 4.5066403602406275 |
|                      | <p align="left"> IDL DLM |<p align="left"> -0.059239285         | <p align="left"> 3.6914216          | <p align="left"> 4.5066502          |


#### Tracing routines

A very important part of `geopack` is `trace` and its subroutines. I will only test on `trace`, since if it works, then the subroutines should all work.

The inputs are:
* Time/dipole tilt: 2001-01-01/02:03:04 UT/`ps = -0.533585131 rad`
* Position: `(xgsm,ygsm,zgsm) = (-5.1,0.3,2.8)`
* Model parameter:
    * T89: `iopt = 2`
    * T96, T01, and T04: `par = [2., -87., 2., -5., 0., 0., ps, xgsm,ygsm,zgsm]`.
* Minimum tracing distance: `r0=1.1`
* Maximum tracing distance: `rlim=10`
* Tracing direction: `dir=-1`
* Internal model: `igrf`

Here are the comparisons of the footpoint location in GSM (I didn't get the Fortran `trace` to work, so the Python and IDL results are compared):

|<p align="left"> Model| <p align="left"> Version | <p align="left"> GSM X (Re) | <p align="left"> GSM Y (Re) | <p align="left"> GSM Z (Re) |
|---|---|---|---|---|
| <p align="left"> T89 | <p align="left"> Python  | <p align="left"> -0.7218581171377333 | <p align="left"> 0.03278201781940832  | <p align="left"> 0.8293646495541395 |
|                      | <p align="left"> IDL DLM | <p align="left"> -0.72185779         | <p align="left"> 0.032780279          | <p align="left"> 0.82936518         |
| <p align="left"> T96 | <p align="left"> Python  | <p align="left"> -0.7289210907095749 | <p align="left"> 0.035354863396548725 | <p align="left"> 0.8230575125223648 |
|                      | <p align="left"> IDL DLM | <p align="left"> -0.72891962         | <p align="left"> 0.035352710          | <p align="left"> 0.82305917         |
| <p align="left"> T01 | <p align="left"> Python  | <p align="left"> -0.726408232521053  | <p align="left"> 0.03894789557265691  | <p align="left"> 0.8251143626438783 |
|                      | <p align="left"> IDL DLM | <p align="left"> -0.72618079         | <p align="left"> 0.039247587          | <p align="left"> 0.82530059         |
| <p align="left"> T04 | <p align="left"> Python  | <p align="left"> -0.7153766644654653 | <p align="left"> 0.024213930362513274 | <p align="left"> 0.8352541020689167 |
|                      | <p align="left"> IDL DLM | <p align="left"> -0.71700103         | <p align="left"> 0.025224728          | <p align="left"> 0.83383019         |

#### Magnetopause models
As the last part, we test `t96_mgnp` and `shuetal_mgnp`. The inputs are:
* Time/dipole tilt: 2001-01-01/02:03:04 UT/`ps = -0.533585131 rad`
* Position: `(xgsm,ygsm,zgsm) = (-5.1,0.3,2.8)`
* Model parameter:
    * T89: `iopt = 2`
    * T96, T01, and T04: `par = [2., -87., 2., -5., 0., 0., ps, xgsm,ygsm,zgsm]`.
* Density: `den = 10`
* Velocity: `vel = 500`
* Bz: `bz = -5`

The model magnetopause locations in GSM are:

|<p align="left"> Model| <p align="left"> Version | <p align="left"> GSM X (Re) | <p align="left"> GSM Y (Re) | <p align="left"> GSM Z (Re) |
|---|---|---|---|---|
| <p align="left"> T96 | <p align="left"> Fortran | <p align="left"> -1.14082575         | <p align="left"> 1.47308505         | <p align="left"> 13.7487936         |
|                      | <p align="left"> Python  | <p align="left"> -1.140826561384304  | <p align="left"> 1.4730846293697084 | <p align="left"> 13.748789874117278 |
|                      | <p align="left"> IDL DLM | <p align="left"> -1.1408265          | <p align="left"> 1.4730847          | <p align="left"> 13.748790          |
| <p align="left"> Shu | <p align="left"> Fortran | <p align="left"> -1.04701376         | <p align="left"> 1.48984993         | <p align="left"> 13.9052658         |
|                      | <p align="left"> Python  | <p align="left"> -1.0470115138905702 | <p align="left"> 1.4898498476086839 | <p align="left"> 13.905265244347715 |
|                      | <p align="left"> IDL DLM | <p align="left"> -1.0470114          | <p align="left"> 1.4898499          | <p align="left"> 13.905265          |

## Summary
As a summary, the above tests demonstrate that the Python `geopack` returns same answers as the Fortran and IDL versions for given inputs. As a powerful language, Python is adopted by the science community and the Python `geopack` is created following the trend. Please contact me (tianx138@umn.edu) for bugs, comments, suggestions, etc. Thansk!

## Appendix

#### A Fortran `hello world`
Create a file named `hello.f90` in your home directory, with the following lines:
```
print *, "Hello World!"
end
```
Then in terminal, compile it
```
> cd ~
> gfortran -o hello hello.f90
```
Then run it
```
> ./hello
Hello World!
```
The last line should be what you see.


#### How to use the Fortran `geopack`

I have downloaded the source code and saved it as `geopack_2005.for`. Now in the terminal, I `cd` to the folder it is saved and compile the code by:
```
$ gfortran -c geopack_2005.for -ffixed-form
```
Note that file extension **matters**: `*.for` is treated as in the fixed form, while `*.f90` is treated as in the free form. Since the Fortran `geopack` and Tsyganenko models are all written in the fixed form, to be save, just use `-ffixed-form` when compiling the codes.

Now I create the archive containing the code
```
$ ar -rcs geopack.a geopack_2005.o
```

Now there should be a file `geopack.a` in the folder. I've created a test code `test_geopack.f90` with the following lines in it:
```
program test_geopack

    implicit none
    real b1,b2,b3
    real r,theta,phi
    
    r = 1.1
    theta = 0.1
    phi = 0.2
    
    ! 2001-01-01/02:03:04, doy = 1
    call recalc(2001, 1, 2, 3, 4)
    call igrf_geo(r,theta,phi, b1,b2,b3)
    
    print *, b1,b2,b3
end
```

To run the test code, in terminal, I type:
```
$ gfortran -o test_geopack test_geopack.f90 geopack.a
$ ./test_geopack
  -42611.9023      -3373.67896      -531.660034    
```

## Test `geopack`

All routines have passed checks except `trace` and its subroutines (need Tsyganenko models).

Test the time 2001-01-01/02:03:04 UT.

* #### `igrf_geo`

Test `(r,theta,phi) = (1.1,0.1,0.2)`, and get similar results for `(b1,b2,b3)`.

In Python:
```
-42611.90231427305 -3373.678578012136 -531.659840375275
```
In Fortran:
```
  -42611.9023      -3373.67896      -531.660034
```

* #### `sun`

Get similar results for `(gst, slong, srasn, sdec)`.

In Python:
```
2.2962409046593333 4.899671433618979 4.915964131105882 -0.4015294967916718
```
In Fortran:
```
   2.29623127       4.89966297       4.91595507     -0.401530176    
```

* #### `sphcar`

Get similar results for `(x,y,z)` with the input of `(r,theta,phi) = (1.1,0.1,0.2)`.
In Python:
```
0.1076277345079813 0.021817221883830864 1.0945045818058283
```
In Fortran:
```
  0.107627742       2.18172241E-02   1.09450459    
```

* #### Coordinate transforms
Get similar results for `(xgsm,ygsm,zgsm) = (6.1,0.3,3.2)`.
In Python:
```
GSM:  6.1 0.3 3.2
GEO:  -6.297744064512124 2.7630452372159606 0.4939642851466979
GSE:  6.1 0.9934129372989612 3.056653326008661
SM:   6.879628646552562 0.3 -0.3474329943073369
MAG:  -4.62194711846858 -5.10459549313622 -0.3474329943073369
GEI:  2.1110295915964987 -6.545193148288552 0.4939642851466979
MAG2: -4.621947118468579 -5.10459549313622 -0.34743299430733826
```
In Fortran:
```
GSM:    6.09999990      0.300000012       3.20000005    
GEO:   -6.29774570       2.76304150      0.493959665    
GSE:    6.09999990      0.993206918       3.05672050    
SM:     6.87962818      0.300000012     -0.347437382    
MAG:   -4.62194300      -5.10459852     -0.347437382    
GEI:    2.11097026      -6.54521179      0.493959665    
MAG2:  -4.62194300      -5.10459852     -0.347436965    
```

* #### `igrf_gsm` and `dip`
Get similar results for `(xgsm,ygsm,zgsm) = (6.1,0.3,3.2)`.
In Python:
```
-33.92890232572948 -0.3727154118150935 89.20745412008773
-34.428340703628955 0.6040536762748749 85.51455538560549
```
In Fortran:
```
  -33.9288025     -0.372708321       89.2075043    
  -34.4282494      0.604061484       85.5146179    
```

* #### `bcarsp` and `bspcar`
Get similar results for `(xgsm,ygsm,zgsm) = (6.1,0.3,3.2)` and `(bxgsm,bygsm,bzgsm)=dip(xgsm,ygsm,zgsm)`.
In Python:
```
9.255344886293766 -91.69235884634486 2.2944776040841424
-34.428340703628955 0.6040536762748749 85.51455538560549
```
In Fortran:
```
   9.25545692      -91.6923752       2.29448080    
  -34.4282455      0.604061246       85.5146255    
```

* #### `shuetal_mgnp` and `t96_mgnp`
Get similar results for `(xgsm,ygsm,zgsm) = (6.1,0.3,3.2)` and `(pdyn,vel,bz) = (10,-1,-5)`, for T96 and Shu+ models.
In Python:
```
7.831191418324835 0.38829185915481085 4.141779830984649 1.9727565560210516
7.019241779564717 0.3599719424615801 3.8397007195901875 1.1215253424796636
```
In Fortran:
```
   7.83119869      0.388290554       4.14176559       1.97275615    
   7.01924562      0.359970629       3.83968616       1.12152016        
```
Similarly, the results agree for `(den,vel,bz) = (10,500,-5)`.
In Python:
```
8.625939375256964 0.437249319747824 4.663992743976789 2.922752445497475
7.802257876894604 0.407946564792566 4.351430024454038 2.057942040368112
```
In Fortran:
```   
   8.62594032      0.437250376       4.66400337       2.92275882    
   7.80225563      0.407947391       4.35143852       2.05794501    
```

* #### `trace` and its subroutines
The `trace` routine is tested using T89 as the external model. The input parameters are:
    * Time: 2001-01-01/02:03:04 UT
    * Initial position: `(xgsm,ygsm,zgsm) = (-5.1,0.3,2.8)`
    * Dipole tilt associated with the time: `-0.533585131 rad`
    * Min radial: `r0 = 1.1 Re`
Get similar footpoint `(xf,yf,zf)` in GSM.
In IDL:
```
     -0.72185779     0.032780279      0.82936518
```
In Python:
```
-0.7218598097728839 0.03277131982950038 0.8293635990155322
```
